## Import required libraries

In [ ]:
import geopandas
import plotly.express as px
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

## Read Ookla dataset
[This tutorial](https://github.com/teamookla/ookla-open-data/blob/master/tutorials/aggregate_by_county_py.ipynb) shows how to download dataset automatically every quarter from [AWS server](https://registry.opendata.aws/speedtest-global-performance/). Since the file is very big and it takes time to download it, to read it and filter required data, I skip this phase here and will be using earlier created file with filtered data for Finland. 

In [ ]:
#world = geopandas.read_file('Data/OrigOoklaData/gps_mobile_tiles.shp')

##print(world.head())
##print(world.columns)


### Filter data data for Finland
Fitering is done using 'quadkey' information. You can check corresponding quadkeys from [Mapbox website](https://labs.mapbox.com/what-the-tile/).

In [ ]:
#finland = world[world['quadkey'].str.startswith('120120033')]
#finland.to_file("finland.shp")
##print(finland.head())
##print(finland.columns)


#### Prepare data for choropleth_mapbox plotting
Since downloading, reading and filtering data takes time. Below I use "finland.shp" file that has already been creted.

In [ ]:
finland = geopandas.read_file('Data/FinlandData/finland.shp') 
finland['avg_d_mbps'] = finland['avg_d_kbps']/1000 #create new column for avg_d_mbps.

#print(finland.head())

##### Quantiles
Quantiles for finland['avg_d_mbps'] in order to plot 'avg_d_mbps' in ranges.
I am pretty sure there is a smarter way to do that, which I have not learnt yet :)

In [ ]:
avg_d_q1 = round(finland['avg_d_mbps'].quantile(.1),1) 
avg_d_q2 = round(finland['avg_d_mbps'].quantile(.2),1)
avg_d_q3 = round(finland['avg_d_mbps'].quantile(.3),1)
avg_d_q4 = round(finland['avg_d_mbps'].quantile(.4),1)
avg_d_q5 = round(finland['avg_d_mbps'].quantile(.5),1)
avg_d_q6 = round(finland['avg_d_mbps'].quantile(.6),1)
avg_d_q7 = round(finland['avg_d_mbps'].quantile(.7),1)
avg_d_q8 = round(finland['avg_d_mbps'].quantile(.8),1)
avg_d_q9 = round(finland['avg_d_mbps'].quantile(.9),1)

bins = [0, avg_d_q1, avg_d_q2, avg_d_q3, avg_d_q4, avg_d_q5, avg_d_q6, avg_d_q7, avg_d_q8, avg_d_q9,finland['avg_d_mbps'].max()]
labels = ['<{}'.format(avg_d_q1),
        '>={} & <{}'.format(avg_d_q1,avg_d_q2),
        '>={} & <{}'.format(avg_d_q2,avg_d_q3),
        '>={} & <{}'.format(avg_d_q3,avg_d_q4),
        '>={} & <{}'.format(avg_d_q4,avg_d_q5),
        '>={} & <{}'.format(avg_d_q5,avg_d_q6),
        '>={} & <{}'.format(avg_d_q6,avg_d_q7),
        '>={} & <{}'.format(avg_d_q7,avg_d_q8),
        '>={} & <{}'.format(avg_d_q8,avg_d_q9),
        '>={}'.format(avg_d_q9)
]

Create new column 'avg_d_mbps_ranges' with corresponding ranges for 'avg_d_mbps'. 
Sort values in finland dataframe to allocate colour on the choropleth_mapbox plot in logical order.

In [ ]:
finland['avg_d_mbps_ranges'] = pd.cut(finland['avg_d_mbps'], bins=bins, labels=labels)
finland = finland.sort_values(by=['avg_d_mbps'])
#print(finland['avg_d_mbps_ranges'].unique())

#finland = finland.head(1000) # is to limit data for running the dashboard on Heroku server.

## Plot data on the map

In [ ]:
fig = px.choropleth_mapbox(finland,
                geojson=finland.geometry,
                locations=finland.index,
                color="avg_d_mbps_ranges",
                color_discrete_sequence= px.colors.sequential.Turbo_r,
                opacity=0.5,
                hover_data = ['avg_d_mbps_ranges','avg_d_mbps','avg_u_kbps','tests','devices'],
                custom_data = ['avg_d_mbps'],
                center = {'lat':60.2825,'lon':24.9271},
                mapbox_style="open-street-map")
#fig.update_geos(fitbounds="locations", visible=False)
fig.show()

## Create dashboard
Dashboard inludes H1, H2, Markdown and Graph elements. 
Create call back for hovering over a map tile. 

In [ ]:
app = dash.Dash()

## Three lines below are form running the dashboard on Heroku server.
#USERNAME_PASSWORD_PAIRS = [['XXXX', 'XXXX']]
#auth = dash_auth.BasicAuth(app,USERNAME_PASSWORD_PAIRS)
#server = app.server

app.layout = html.Div([
    html.H1('Speedtest by Ookla Mobile Network'),
    html.H2('Performance Map Tiles'),
    dcc.Markdown('''
                #### This dashbord was built using [Ookla Open Dataset](https://registry.opendata.aws/speedtest-global-performance/).
                ##### Used data shows mobile (cellular) network performance metrics in zoom level 16 web mercator tiles (approximately 610.8 meters by 610.8 meters at the equator). 
            '''),

    dcc.Graph(id='map',figure=fig),
    html.Div([
        html.H3(id='counter')
    ],style={'display':'inline-block'})  
])


@app.callback(Output('counter', 'children'),
              [Input('map', 'hoverData')])
def find_counter(hoverData):
    #pts = len(selectedData['points'])
    #rng_or_lp = list(selectedData.keys())
    #rng_or_lp.remove('lassoPoints')
    
    mean = hoverData['points'][0]['customdata'][0]
   # data = selectedData['points']['customdata'[2]]
    #rng_or_lp = list(selectedData.keys())
    #rng_or_lp.remove('points')
    #avg_thp = mean(selectedData[rng_or_lp[0]]['customdata'[1]])
    #return 'Counter = {:.2f}'.format(mean)
    return 'Avg_d_mbps in the tile you hover over is {} Mpbs.'.format(mean)

if __name__ == '__main__':
    app.run_server(use_reloader=False)  # Turn off reloader if inside Jupyter
